# Revised case normalization for Hirslanden Aarau 2017

- No matches were found in this hospital



In [1]:
import sys

from loguru import logger

sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.py.global_configs import *
from src.revised_case_normalization.py.normalize import normalize
from src.service import bfs_cases_db_service as bfs_db
from src.service.bfs_cases_db_service import session, get_hospital_year_cases

In [2]:
file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
         'Hirslanden Aarau',
        '2017',
        ['Aarau_2017'])

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Aarau', year='2017', sheets=['Aarau_2017'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0)

2022-10-19 07:52:51.109 | INFO     | src.revised_case_normalization.py.normalize:normalize:46 - Read 14 cases for Hirslanden Aarau 2017
2022-10-19 07:52:51.112 | INFO     | src.revised_case_normalization.py.normalize:normalize:69 - TYPES:
case_id              int64
patient_id          object
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object
2022-10-19 07:52:51.116 | INFO     | src.utils.dataframe_utils:validate_icd_codes:68 - Validated ICD codes in 'added_icds' and stored them into 'added_icds': All rows contain valid codes
2022-10-19 07:52:51.117 | INFO     | src.utils.dataframe_utils:validate_icd_codes:68 - Validated ICD codes in 'removed_icds' and stored them into 'removed_icds': All rows contain val

In [4]:
df_revised_case_d2d

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,41206174,1176423,W,72,5,3,I21C,T840,T840,M200,[E1140],[],"[80051D, 815255]",[009031::20170106]
1,41230776,1659346,W,67,19,3,B72C,G001,G001,M100,[B370],[],[],[]
2,41281042,1823347,M,67,6,3,F14B,E1150,E1150,M100,[F101],[],[],[]
3,41386032,1051316,M,74,10,3,I08C,S7211,S7211,M200,[L8914],[],[],[]
4,41418378,22136564,M,86,8,3,G18B,C182,C182,M200,[E1191],[E1190],[],[]
5,41247420,1251372,W,63,8,3,I09D,M4216,M4216,M200,[M4196],[],[],[]
6,41477047,1536048,W,74,6,3,I66B,M315,M315,M100,[K3182],[K5581],[],[]
7,41280551,0022114393,W,84,7,3,F62C,I5012,I5012,M100,"[F023, F338, F051]","[F03, F328]",[],[]
8,41457913,0001193700,W,92,5,3,E71B,C341,C341,M100,[R15],[],[],[]
9,41523346,0002096726,W,77,9,3,L60D,N1783,N1783,M100,[A099],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [5]:
cases_in_db = get_hospital_year_cases(file_info.hospital_name_db, file_info.year)[VALIDATION_COLS]

num_cases_in_db = cases_in_db.shape[0]
if num_cases_in_db == 0:
    raise ValueError(f"There is no data for the hospital '{file_info.hospital_name_db}' in {file_info.year}")
else:
    logger.info(f"Read {num_cases_in_db} rows from the DB, for the hospital '{file_info.hospital_name_db}' in {file_info.year}")

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused
	Is the server running on host "aimedic-bfs-db.ckwggxrzoh8y.eu-central-1.rds.amazonaws.com" (172.32.2.239) and accepting
	TCP/IP connections on port 5432?

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# get the case_id from revised_case
df_revised_case_d2d['case_id_num'] = df_revised_case_d2d[CASE_ID_COL].astype(int)
cases_in_db['case_id_num'] = cases_in_db[CASE_ID_COL].astype(int)

# TODO Join tables

sys.exit(0)

In [ ]:
# match to the database and retrieve data from database using revised
revised_case_db = bfs_db.get_bfs_cases_by_ids_no_pad0(revised_case_id_stripped)
revised_case_db.head()

In [ ]:
sys.exit(0)

In [ ]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()

In [ ]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()
print(revised_case_db.shape)

In [ ]:
# create a new column for case_id_stripped with '0' for joining to cases from database

df_revised_case_d2d['case_id_stripped']=  df_revised_case_d2d['case_id'].apply(lambda x: x.lstrip('0')) 
df_revised_case_d2d.head()
print(df_revised_case_d2d.shape)

In [ ]:
# delete duplicated cases in the revised cases

df_revised_case_d2d['case_id_stripped'].apply(tuple) # make sure the drop_duplicates works

df_revised_case_d2d.drop_duplicates(subset=['case_id_stripped'], keep='last', inplace=True)
assert len(revised_case_id_stripped) == df_revised_case_d2d.shape[0] # make sure all cases are deleted


# Forming the normalized dataframe: Merge the selected revised case data with the matched from bfs data
  If there are multiple cases were found based on case_id, patient_id will be used to match once more  

In [ ]:
# Check if there are multiple cases were found based on case_id, patient_id will be used to match once more
if revised_case_db.shape[0] <= len(revised_case_id):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_db.merge(revised_case_d2d_subset, how ='left', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_db.merge(revised_case_d2d_subset, how ='left', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
revised_case_normalized.head()

In [ ]:
# Choose column to export to a csv file
df_revised_case_normalized = revised_case_normalized[COLS_TO_OUTPUT]
assert len(df_revised_case_normalized.columns) == len(COLS_TO_OUTPUT)
df_revised_case_normalized.head()

# Export the normalized revised cases with the neccessary columns to a csv file


In [ ]:

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + '.csv'

df_revised_case_normalized.to_csv(output_path)


# Report

A small summary of:
- how many cases could be matched in the DB, and based on which columns
- how many cases were “correctly” revised, in terms of upcoding (still need to do after the `grouper` is available for python)

In [ ]:
if revised_case_db.shape[0] <= len(revised_case_id):
    print(f'Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )
else:
    print(f'Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_patient_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )

In [ ]:
print('All the normalized revised cases are exported to here: {}'.format(output_path))